##### 저장된 모델 활용
- 모델 파일 종류
    * 가중치 및 절편 저장 파일 => 동일한 구조의 모델 인스턴스 생성 후 사용 가능
    * 모델 전체 저장 파일 => 바로 로딩 후 사용 가능

In [1]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from classes import CancerModel

import pandas as pd

In [2]:
# class CancerModel(nn.Module):
#     def __init__(self,in_out=5,perceptrons = []) :
#         super().__init__()
#         self.i_layer = nn.Linear(14,perceptrons[0] if len(perceptrons)>0 else in_out)
        
#         self.h_layers = nn.ModuleList()
#         for idx in range(len(perceptrons)-1) :
#             self.h_layers.append(nn.Linear(perceptrons[idx], perceptrons[idx+1]))

#         self.o_layer = nn.Linear(perceptrons[-1] if len(perceptrons)>0 else in_out,1)

#     def forward(self, x):
#         # 입력층
#         y = F.relu(self.i_layer(x))

#         # 은닉층
#         for layer in self.h_layers:
#             y = F.relu(layer(y))
        
#         # 출력층
#         return F.sigmoid(self.o_layer(y))

In [3]:
# # Dropout 실시

# class CancerModel(nn.Module):
#     def __init__(self,in_out=5,perceptrons = []) :
#         super(CancerModel, self).__init__()
#         self.i_layer = nn.Linear(14,perceptrons[0] if len(perceptrons)>0 else in_out)
        
#         self.h_layers = nn.ModuleList()
#         for idx in range(len(perceptrons)-1) :
#             self.h_layers.append(nn.Linear(perceptrons[idx], perceptrons[idx+1]))
#         self.dropout_prob = 0.5

#         self.o_layer = nn.Linear(perceptrons[-1] if len(perceptrons)>0 else in_out,1)

#     def forward(self, x):
#         # 입력층
#         y = F.relu(self.i_layer(x))

#         # 은닉층
#         for layer in self.h_layers:
#             y = F.relu(layer(y))
#             y = F.dropout(y, p=self.dropout_prob)
#         # 출력층
#         return F.sigmoid(self.o_layer(y))

In [4]:
## 모델 파일 관련
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../DeepLearning/models/project/'
# 저장 파일명
SAVE_FILE = SAVE_PATH+'model_train_wb.pth'

# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL = SAVE_PATH+'model_all.pth'

In [5]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH) : os.makedirs(SAVE_PATH)   # 폴더 / 폴더 / ...  하위폴더까지 생성

#### 모델 로딩

In [6]:
# cancerModel = torch.load(SAVE_MODEL, weights_only=False)

In [7]:
# summary(cancerModel)

In [8]:
def prerocessing():
    lungDF = pd.DataFrame([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]],columns=['age', 'gender', 'diagnosis_date', 'cancer_stage', 'beginning_of_treatment_date',  'family_history', 
 'smoking_status', 'bmi', 'cholesterol_level', 'hypertension', 'asthma', 'cirrhosis', 'other_cancer', 'treatment_type', 'end_treatment_date'])
    data = input('데이터를 입력하세요').split(',')
    lungDF.loc[0] = data
    lungDF['age'] = (lungDF['age'].astype('float64')).astype('int')
    lungDF['bmi'] = lungDF['bmi'].astype('float64')
    lungDF['cholesterol_level'] = lungDF['cholesterol_level'].astype('int64')
    lungDF['hypertension'] = lungDF['hypertension'].astype('int64')
    lungDF['asthma'] = lungDF['asthma'].astype('int64')
    lungDF['cirrhosis'] = lungDF['cirrhosis'].astype('int64')
    lungDF['other_cancer'] = lungDF['other_cancer'].astype('int64')
    lungDF['beginning_of_treatment_date'] = pd.to_datetime(lungDF['beginning_of_treatment_date'])
    lungDF['end_treatment_date'] = pd.to_datetime(lungDF['end_treatment_date'])
    lungDF['diagnosis_date'] = pd.to_datetime(lungDF['diagnosis_date'])
    lungDF['cancer_stage'] = lungDF['cancer_stage'].replace({'Stage I':1,'Stage II':2, 'Stage III':3, 'Stage IV':4})
    lungDF['gender'] = lungDF['gender'].replace({'Male':0, 'Female':1})
    lungDF['family_history'] = lungDF['family_history'].replace({'No':0, 'Yes':1})
    lungDF['smoking_status'] = lungDF['smoking_status'].replace({'Never Smoked':0,'Passive Smoker':1,'Former Smoker':2,'Current Smoker':3})
    lungDF['treatment_type'] = lungDF['treatment_type'].replace({'Chemotherapy':0,'Combined':1,'Radiation':2,'Surgery':3})
    lungDF['start_days'] = (lungDF['beginning_of_treatment_date']-lungDF['diagnosis_date']).dt.days
    lungDF['treatment_days'] = (lungDF['end_treatment_date'] - lungDF['beginning_of_treatment_date']).dt.days
    cancerDF = lungDF.drop(['diagnosis_date','beginning_of_treatment_date','end_treatment_date'], axis=1)

    return cancerDF.loc[0].to_list()

#### 예측

In [9]:
# data = prerocessing()
# dataTS = torch.FloatTensor(data).reshape(1,-1)

In [10]:
# cancerModel.eval()

# with torch.no_grad():
#     pre_val = cancerModel(dataTS)

In [11]:
# pre_val

In [12]:
# 'Survived : True' if (pre_val>0.2182).item() else 'Survived : False'

In [13]:
def detecting():
    cancerModel = CancerModel()
    data = prerocessing()
    dataTS = torch.FloatTensor(data).reshape(1,-1)

    cancerModel.eval()

    with torch.no_grad():
        pre_val = cancerModel(dataTS)

    result = 'Survived : True' if (pre_val>0.2182).item() else 'Survived : False'
    return result


In [14]:
result = detecting()
print(result)

Survived : True
